In [1]:
import numpy as np
from sympy import * 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import copy
from numpy.linalg import norm, solve
from mpl_toolkits import mplot3d
sns.set_theme()

# u(x, t)_t = κ*u(x, t)_xx + f (x, t),
# κ = const > 0, 0 < x < a, 0 < t <= T
# u(x, 0) = µ(x), 0 <= x <= a;
# u(0, t) = µ1(t), u(a, t) = µ2(t), 0 <= t <= T.

In [2]:
def solve_equasion(m):
    def implicit():
        def tridiagonal(k):
            alpha_temp = [0 for _ in range(n+1)]
            beta_temp = [0 for _ in range(n+1)]
            gamma_temp = [0 for _ in range(n+1)]
            gamma_temp[0] = 1
            alpha_temp[0] = 0
            beta_temp[0] = mu1(t[k])
            for i in range(1, n):
                a_i = koef/(h*h)
                b_i = -2*koef/(h*h) - 1/tau
                c_i = koef/(h*h)
                d_i = -u[k-1, i]/tau - f(t[k], x[i])
                gamma_temp[i] =  b_i + a_i*alpha_temp[i-1]
                alpha_temp[i] = - c_i / gamma_temp[i]
                beta_temp[i] = (d_i - a_i * beta_temp[i-1])/gamma_temp[i]
            gamma_temp[n] = 1
            beta_temp[n] = mu2(t[k])
            u[k, n] = beta_temp[n]
            for i in range(n-1, -1, -1):
                u[k, i] = alpha_temp[i]*u[k, i+1] + beta_temp[i]
        u = np.zeros((m+1, n+1))
        for i in range(n+1):
            u[0, i] = mu(x[i])
        for k in range(1, m+1):
            tridiagonal(k)
        return u

    def explicit():
        u = np.zeros((m+1, n+1))
        for i in range(m+1):
            u[i, 0] = mu1(t[i])
            u[i, n] = mu2(t[i])
        for i in range(n+1):
            u[0, i] = mu(x[i])
        for i in range(1, m+1):
            for j in range(1, n):
                u[i, j] = u[i-1, j] + tau*koef*(u[i-1, j+1]-2*u[i-1, j] + u[i-1, j-1])/(h*h) + tau*f(t[i-1], x[j])
        return u
    
    h = a/n
    tau = T/m
    x = [i*h for i in range(n+1)]
    t = [i*tau for i in range(m+1)]
    u1 = explicit()
    u2 = implicit()
    res = np.zeros((m+1, n+1))
    for i in range(m+1):
        for j in range(n+1):
            res[i, j] = answer(t[i], x[j])
    return norm(u1-res), norm(u2-res)
#     print(f'Норма невязки решения по явной схеме: {norm(u1-res)}\nНорма невязки по неявной схеме: {norm(u2-res)}')

In [3]:
a = 5
T = 8
koef = 3
mu = lambda x: 3*x*x + 5
mu1 = lambda t: 4*t*t + 5
mu2 = lambda t: 4*t*t + 80
def f(t, x):
    return 8*t-18
def answer(t, x):
    return 3*x*x + 4*t*t + 5

In [4]:
res = []
for i in range(2, 50, 5):
    n = i
    m = round(n*n*4*koef*T/(a*a))
    res.append(((n, m), solve_equasion(m)))

In [5]:
print('Выполнено условие на устойчивость явной схемы\n')
print('(n, m), Норма невязки неявной схемы, Норма невязки явной схемы\n')
for i in res:
    print(f'{i[0]}\n{i[1][1]:<7.3f}{i[1][0]:<7.3f}\n')

Выполнено условие на устойчивость явной схемы

(n, m), Норма невязки неявной схемы, Норма невязки явной схемы

(2, 15)
7.659  8.137  

(7, 188)
4.299  4.319  

(12, 553)
3.288  3.293  

(17, 1110)
2.763  2.765  

(22, 1859)
2.429  2.430  

(27, 2799)
2.193  2.194  

(32, 3932)
2.015  2.015  

(37, 5257)
1.874  1.874  

(42, 6774)
1.759  1.759  

(47, 8483)
1.662  1.663  



In [6]:
res = []
for i in range(2, 50, 5):
    n = i
    m = round(n*n*2*koef*T/(2*a*a))
    res.append(((n, m), solve_equasion(m)))

<ipython-input-2-87a0a91c3f70>:39: RuntimeWarning: overflow encountered in double_scalars
  u[i, j] = u[i-1, j] + tau*koef*(u[i-1, j+1]-2*u[i-1, j] + u[i-1, j-1])/(h*h) + tau*f(t[i-1], x[j])
<ipython-input-2-87a0a91c3f70>:39: RuntimeWarning: invalid value encountered in double_scalars
  u[i, j] = u[i-1, j] + tau*koef*(u[i-1, j+1]-2*u[i-1, j] + u[i-1, j-1])/(h*h) + tau*f(t[i-1], x[j])


In [7]:
print('Не выполнено условие на устойчивость явной схемы\n')
print('(n, m), Норма невязки неявной схемы, Норма невязки явной схемы\n')
for i in res:
    print(f'{i[0]}\n{i[1][1]:<7.3f}{i[1][0]:<7.3f}\n')

Не выполнено условие на устойчивость явной схемы

(n, m), Норма невязки неявной схемы, Норма невязки явной схемы

(2, 4)
14.685 21.976 

(7, 47)
8.580  363805694785197.250

(12, 138)
6.576  7859974619324582422874997655496058967033747469519069445619712.000

(17, 277)
5.529  445869994382596786906165425399732290527652971844093299220708794686746819483457392654911388427310212982696922426698415210496.000

(22, 465)
4.856  inf    

(27, 700)
4.385  nan    

(32, 983)
4.029  nan    

(37, 1314)
3.747  nan    

(42, 1693)
3.518  nan    

(47, 2121)
3.325  nan    

